In [448]:
from copy import copy

In [694]:
VAR_INDEX = 0

# Abstract class Formula
class Formula(): 
    def __init__(self, args, negated=False):
        self.negated = negated
        self.args = args  
    def __neg__(self): 
        new_object = copy(self)
        new_object.negated = not new_object.negated
        return new_object 
    def __add__(self, other): return Disjunction(self, other)
    def __mul__(self, other): return Conjunction(self, other)
    def __rshift__(self, other): return Implication(self, other)
    def __mod__(self, other): return Equivalence(self, other)
    
    

# Abstract class Connective 
class Connective(Formula): 
    def __init__(self, left, right):
        super().__init__([left, right], False)

# Conjunction
class Conjunction(Connective):
    pass

# Disjunction
class Disjunction(Connective): 
    pass

# Implication
class Implication(Formula):
    def __init__(self, left, right):
        super().__init__([left, right], False)

# Equivalence
class Equivalence(Formula): 
    def __init__(self, left, right):
        super().__init__([left, right], False)

# Variable
class Var(Formula):
    def __init__(self, name, specific_negated=False):
        super().__init__(None, negated=specific_negated)        
        global VAR_INDEX
        self.index = VAR_INDEX
        VAR_INDEX += 1
        self.name = name

In [705]:
def remove_imp_eq(formula):
    if type(formula) == Var:
        return formula
    
    if type(formula) == Conjunction:
        expr = remove_imp_eq(formula.args[0]) * remove_imp_eq(formula.args[1])
        expr.negated = formula.negated
        return expr
    
    if type(formula) == Disjunction:
        expr = remove_imp_eq(formula.args[0]) + remove_imp_eq(formula.args[1])
        expr.negated = formula.negated
        return expr
    
    if type(formula) == Implication:
        expr = remove_imp_eq(-(formula.args[0])) + remove_imp_eq(formula.args[1])
        expr.negated = formula.negated
        return expr
    
    if type(formula) == Equivalence:
        expr = remove_imp_eq(formula.args[0]) * remove_imp_eq(formula.args[1]) + \
               remove_imp_eq(-(formula.args[0])) * remove_imp_eq(-(formula.args[1]))
        expr.negated = formula.negated
        return expr

In [706]:
a = Var('a')
b = Var('b')

In [707]:
f = (a % a) >> (b % b)
r = remove_imp_eq(f)
print(r.args[0].name, r.args[0].negated)
print(r.args[1].name, r.args[1].negated)
print(type(r))

impl
eq
eq


AttributeError: 'Disjunction' object has no attribute 'name'